# Technology focus: Stereo-seq

Note: this notebook is currently uploaded without plots because it is run on non-public data.

In [ ]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [ ]:
import os

## Loading the data

We use the reader `stereoseq()` from `spatialdata-io` to parse the data.

In [ ]:
WRITE = True
WRITE = False

In [ ]:
from spatialdata_io import stereoseq

# the data is not currently public
stereoseq_data_path = "stereoseq_data/pipeline output/result"
assert os.path.exists(stereoseq_data_path)

if WRITE:
    sdata = stereoseq(stereoseq_data_path)
    sdata

Let's save the data to Zarr and reload it (so that the data is accessed performantly).

In [ ]:
import shutil

import spatialdata as sd
import spatialdata_plot  # noqa: F401

output_data_path = "stereoseq_data/data.zarr"

if WRITE:
    # please use rmtree with care
    if os.path.isdir(output_data_path):
        shutil.rmtree(output_data_path)

    sdata.write(output_data_path)

sdata = sd.read_zarr(output_data_path)

## Plotting the images

In [ ]:
sdata.pl.render_images("ssDNA_C01337B3_regist").pl.show()

Let's introduce a lambda function for defining a lazy crop of the image data; we will use it to plot a subset of the data.

In [ ]:
images_names = list(sdata.images.keys())
images_names

In [ ]:
def crop(sdata):
    return sd.bounding_box_query(
        # let's subset the data to consider only the images since we are going to plot only them
        sdata.subset(images_names),
        min_coordinate=[9000, 11000],
        max_coordinate=[10000, 12000],
        axes=("x", "y"),
        target_coordinate_system="global",
    )

In [ ]:
crop(sdata).pl.render_images("ssDNA_C01337B3_regist").pl.show()

In [ ]:
crop(sdata).pl.render_images("ssDNA_C01337B3_mask").pl.show()

## Plotting the cell expression data

The cell geometries are stored in `sdata.shapes['cells_polygons']`; and approximations of them as circles is stored in `sdata.shapes['cells_circles']`.

The cell gene expression data is stored in the associated table `sdata.tables['cells_table']`.

Let's have a look at the table.

In [ ]:
sdata["cells_table"]

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("dark_background")
sdata.pl.render_shapes("cells_circles", color="clusterID").pl.show()
plt.style.use("default")

In [ ]:
sdata_crop = sd.bounding_box_query(
    sdata,
    min_coordinate=[9000, 11000],
    max_coordinate=[10000, 12000],
    axes=("x", "y"),
    target_coordinate_system="global",
)

In [ ]:
plt.style.use("dark_background")
sdata_crop.pl.render_shapes("cells_circles", color="clusterID").pl.show()
plt.style.use("default")

To render the cells shapes we need first to assign the table to the shapes object.

In [ ]:
sdata_crop["cells_table"].obs["region"] = "cells_polygons"
sdata_crop["cells_table"].obs["region"] = sdata_crop["cells_table"].obs["region"].astype("category")
sdata_crop.set_table_annotates_spatialelement("cells_table", "cells_polygons")

In [ ]:
# can't make this plot because of this bug: https://github.com/scverse/spatialdata-plot/issues/266
sdata_crop.pl.render_shapes("cells_polygons", color="clusterID").pl.show()

## Plotting the bin data

The object also contains the binned gene expression data at various resolutions (subcellular and supercellular). For each bins we store both the geometries (e.g. `sdata.points['bin20_genes']`) and the binned gene expression (e.g. `sdata.tables['bin20_table']`).

Note that even if the bin geometries should be represented as squares (=shapes element), we use points for performance reasons as the data is very large. The user can easily construct a shapes element from the points element if needed.

Let's explore one of the the bin annotation table.

In [ ]:
sdata["bin20_table"]

In [ ]:
sdata["bin20_table"].var_names

In [ ]:
gene = "0610010K14Rik"

In [ ]:
# bug: cropping points doesn't filter the associated table
# https://github.com/scverse/spatialdata/issues/567
# let's have a workaround for this

sdata_crop["bin20_table"] = sdata["bin20_table"]
queried_points = sdata_crop["bin20_genes"].index.compute()
sdata_crop["bin20_table"] = sdata_crop["bin20_table"][queried_points].copy()

In [ ]:
# can't make this plot because of this bug: https://github.com/scverse/spatialdata-plot/issues/265
sdata_crop.pl.render_points("bin20_genes", color=gene).pl.show()